# WEST C3b ICRH pulses database

In this notebook we generate a database of the pulses in which ICRH has been used. 

In theory, the IRFM GED could be used for that... In theory.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
import pywed as pw
import numpy as np

from control_room import *
import matplotlib.pyplot as plt 
from pulse_database import PulseDB

# Listing the pulses

Make a list of all the pulses to analyze:

In [3]:
hdf5_filename = 'databases/WEST_C3b_ICRH_pulse_data.hdf5'
db = PulseDB(hdf5_filename)

In [4]:
# Below we add the pulse number for which ICRH has been used (>100k W).
db.update_pulse_list()

Going from 54178 to 54178


In [5]:
db.pulse_list

[53566,
 53567,
 53579,
 53580,
 53581,
 53582,
 53583,
 53584,
 53588,
 53589,
 53590,
 53591,
 53594,
 53596,
 53604,
 53608,
 53609,
 53610,
 53630,
 53632,
 53633,
 53636,
 53637,
 53638,
 53639,
 53640,
 53643,
 53672,
 53674,
 53675,
 53676,
 53677,
 53678,
 53679,
 53680,
 53681,
 53682,
 53683,
 53684,
 53686,
 53687,
 53688,
 53689,
 53690,
 53691,
 53693,
 53694,
 53695,
 53696,
 53698,
 53699,
 53700,
 53701,
 53702,
 53703,
 53704,
 53705,
 53706,
 53730,
 53733,
 53772,
 53773,
 53774,
 53775,
 53776,
 53777,
 53778,
 53779,
 53780,
 53781,
 53782,
 53784,
 53787,
 53790,
 53791,
 53792,
 53793,
 53844,
 53868,
 53869,
 53871,
 53872,
 53875,
 53876,
 53877,
 53879,
 53889,
 53897,
 53898,
 53899,
 53903,
 53905,
 53906,
 53907,
 53917,
 53921,
 53923,
 53930,
 53931,
 53932,
 53934,
 53935,
 53970,
 53971,
 53973,
 53975,
 53982,
 53983,
 53984,
 54001,
 54004,
 54005,
 54006,
 54087,
 54089,
 54090,
 54091,
 54092,
 54094,
 54102,
 54105,
 54105,
 54105,
 54177,
 54178,


In [7]:
# save the pulse list in a text file
np.savetxt('WEST_C3b_IC_pulse_list.txt', db.pulse_list, fmt='%i')

# Filling the database

In [12]:
# Adding new signals to the database
#db.delete_signals(['LH_Positions', 'IC_Positions', 'IC_Frequencies'])
signal_names = ['IC_Capa_Q1_left_upper', 'IC_Capa_Q1_left_lower', 'IC_Capa_Q1_right_upper', 'IC_Capa_Q1_right_lower',
                'IC_Capa_Q2_left_upper', 'IC_Capa_Q2_left_lower', 'IC_Capa_Q2_right_upper', 'IC_Capa_Q2_right_lower',]
#signal_names = ['IC_Voltage_left_upper_Q1', 'IC_Voltage_left_lower_Q1', 
#                'IC_Voltage_right_upper_Q1', 'IC_Voltage_right_lower_Q1',
#                'IC_Voltage_left_upper_Q2', 'IC_Voltage_left_lower_Q2', 
#                'IC_Voltage_right_upper_Q2', 'IC_Voltage_right_lower_Q2']

signal_names = ['IC_Frequencies', 
                'IC_P_Q1', 'IC_P_Q2', 'IC_P_tot', 'LH_P_tot',
                'IC_Rc_Q1_avg', 'IC_Rc_Q2_avg']

#signal_names = ['Rext_upper', 'Rext_median', 'Rext_lower', 'Zgeo']

#signal_names = ['LPA', 'IC_Positions', 'LH_Positions']

#signal_names = ['nl', 'Valve7', 'Valve9','Valve11','Valve21']

#signal_names = ['Ip']

#TODO current and capacitor values

signal_names = ['IC_Capa_Q1_left_upper', 'IC_Capa_Q1_left_lower', 'IC_Capa_Q1_right_upper', 'IC_Capa_Q1_right_lower',
                'IC_Capa_Q2_left_upper', 'IC_Capa_Q2_left_lower', 'IC_Capa_Q2_right_upper', 'IC_Capa_Q2_right_lower']

signal_names = ['IC_Vacuum_Q1_left', 'IC_Vacuum_Q1_right', 'IC_Vacuum_Q2_left', 'IC_Vacuum_Q2_right']

for pulse in db.pulse_list:
    db.add_signals(int(pulse), signal_names)

Getting IC_Vacuum_Q1_left for #53566
Getting IC_Vacuum_Q1_right for #53566
Getting IC_Vacuum_Q2_left for #53566
Getting IC_Vacuum_Q2_right for #53566
Getting IC_Vacuum_Q1_left for #53567
Getting IC_Vacuum_Q1_right for #53567
Getting IC_Vacuum_Q2_left for #53567
Getting IC_Vacuum_Q2_right for #53567
Getting IC_Vacuum_Q1_left for #53579
Getting IC_Vacuum_Q1_right for #53579
Getting IC_Vacuum_Q2_left for #53579
Getting IC_Vacuum_Q2_right for #53579
Getting IC_Vacuum_Q1_left for #53580
Getting IC_Vacuum_Q1_right for #53580
Getting IC_Vacuum_Q2_left for #53580
Getting IC_Vacuum_Q2_right for #53580
Getting IC_Vacuum_Q1_left for #53581
Getting IC_Vacuum_Q1_right for #53581
Getting IC_Vacuum_Q2_left for #53581
Getting IC_Vacuum_Q2_right for #53581
Getting IC_Vacuum_Q1_left for #53582
Getting IC_Vacuum_Q1_right for #53582
Getting IC_Vacuum_Q2_left for #53582
Getting IC_Vacuum_Q2_right for #53582
Getting IC_Vacuum_Q1_left for #53583
Getting IC_Vacuum_Q1_right for #53583
Getting IC_Vacuum_Q2_left

In [9]:
db.list_signal(53702)

['IC_Capa_Q1_left_lower',
 'IC_Capa_Q1_left_upper',
 'IC_Capa_Q1_right_lower',
 'IC_Capa_Q1_right_upper',
 'IC_Capa_Q2_left_lower',
 'IC_Capa_Q2_left_upper',
 'IC_Capa_Q2_right_lower',
 'IC_Capa_Q2_right_upper',
 'IC_Frequencies',
 'IC_P_Q1',
 'IC_P_Q2',
 'IC_P_tot',
 'IC_Positions',
 'IC_Rc_Q1_avg',
 'IC_Rc_Q2_avg',
 'IC_Voltage_left_lower_Q1',
 'IC_Voltage_left_lower_Q2',
 'IC_Voltage_left_upper_Q1',
 'IC_Voltage_left_upper_Q2',
 'IC_Voltage_right_lower_Q1',
 'IC_Voltage_right_lower_Q2',
 'IC_Voltage_right_upper_Q1',
 'IC_Voltage_right_upper_Q2',
 'Ip',
 'LH_P_tot',
 'LH_Positions',
 'LPA',
 'Rext_lower',
 'Rext_median',
 'Rext_upper',
 'Valve10',
 'Valve11',
 'Valve21',
 'Valve7',
 'Valve9',
 'Zgeo',
 'nl']